In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
from sk import predict
from glob import glob
from tqdm import tqdm

In [3]:
import malaya
import malaya_speech
import tensorflow as tf
malaya.__version__, tf.__version__

2021-10-07 22:51:42.909038: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
/home/ubuntu/.local/lib/python3.8/site-packages/malaya_boilerplate/frozen_graph.py:24: UserWarning: Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0
  warnings.warn(


('4.7', '2.5.0')

In [4]:
# !pip3 install tensorflow-text==2.5 --no-deps
# !pip3 install tensorflow_hub

In [5]:
# super_super_tiny = malaya.true_case.transformer(model = 'super-tiny-t5', device = 'cpu:0')

In [6]:
aligner = malaya_speech.force_alignment.deep_aligner(model = 'conformer-transducer', device = 'cpu:0')

2021-10-07 22:51:51.007549: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-07 22:51:51.013536: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-10-07 22:51:51.056345: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-07 22:51:51.056381: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: huseincomel-desktop
2021-10-07 22:51:51.056389: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: huseincomel-desktop
2021-10-07 22:51:51.056483: I tensorflow/stream_executor/cuda/cuda_diagnos

In [7]:
wavs = glob('turki-short/*/*.wav')
len(wavs)

6133

In [8]:
def put(alignment, length, min_threshold = 0.5, min_threshold_fullstop = 0.8):
    r = []
    for no, row in enumerate(alignment):
        if no > 0:
            if alignment[no]['start'] - alignment[no-1]['end'] >= min_threshold:
                r.append(',')

        r.append(row['text'])
    if length - alignment[-1]['end'] >= min_threshold_fullstop:
        r.append('.')
    else:
        r.append(',')
    return r

In [14]:
results = {}
for wav in tqdm(wavs):
    with open(wav.replace('-wav-', '-text-') + '.txt') as fopen:
        data = fopen.read().lower()
#     data = predict(wav, decoder = 'v1')['texts'][0]
    y, sr = malaya_speech.load(wav)
    result = aligner.predict(y, data)
    if ' '.join([r['text'] for r in result['words_alignment']]) != data:
        continue
    o = put(result['words_alignment'], len(y) / sr)
    results[wav] = ' '.join(o)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6133/6133 [16:42<00:00,  6.12it/s]


In [16]:
results

{'turki-short/output-wav-turki/orang-turk-memperjuangkan-hak-mereka-3.mp3-740.wav': 'apa yang dikenali di turki , sebagai gaya seni bina kebangsaan ,',
 'turki-short/output-wav-turki/ilusi-kuasa-1.mp3-302.wav': 'akibat kekurangan zat makanan pada masa perang , banyak kematian disebabkan oleh penyakit tifus ,',
 'turki-short/output-wav-turki/orang-turk-memperjuangkan-hak-mereka-3.mp3-866.wav': 'kutu pak nasionalis dapat sedih ,',
 'turki-short/output-wav-turki/ilusi-kuasa-1.mp3-340.wav': 'namun begitu ,',
 'turki-short/output-wav-turki/janji-tinggal-janji-2.mp3-471.wav': 'menurut seorang sarjana timur tengah yang ter ,',
 'turki-short/output-wav-turki/orang-turk-memperjuangkan-hak-mereka-3.mp3-1066.wav': 'bersidang sebelum memberi persetujuan kepada semua tuntutan berikat tadi ,',
 'turki-short/output-wav-turki/orang-turk-memperjuangkan-hak-mereka-3.mp3-596.wav': 'orang gerik tempatan pula enggan terlibat dengan pilihan raya ,',
 'turki-short/output-wav-turki/revolusi-barat-di-timur-4.m

In [18]:
import IPython.display as ipd
ipd.Audio('turki-short/output-wav-turki/ilusi-kuasa-1.mp3-473.wav')

In [19]:
len(results), len(wavs)

(5485, 6133)

In [20]:
import json
with open('comma-fullstop-turki-short.json', 'w') as fopen:
    json.dump(results, fopen)